# Data Cleaned

In [88]:
import numpy as np
import pandas as pd

In [89]:
df_vacc = pd.read_csv('child_vaccination_data.csv')
num_rows_original = df_vacc.shape[0]
df_vacc.shape

(118423, 10)

In [90]:
df_vacc.head()

,Vaccine,Dose,Geography Type,Geography,Birth Year/Birth Cohort,Dimension Type,Dimension,Estimate (%),95% CI (%),Sample Size
0,Influenza,NaN,States/Local Areas,Tennessee,2014-2017,Poverty,>400% FPL,64.2,55.3 to 73.1,189.0
1,Hep A,≥1 Dose,States/Local Areas,Tennessee,2016-2019,Race and Ethnicity,Hispanic,86.4,76.8 to 95.9,187.0
2,PCV,≥4 Doses,States/Local Areas,Tennessee,2016-2019,Insurance Coverage,Private Insurance Only,89.3,85.4 to 93.2,612.0
3,Influenza,NaN,States/Local Areas,Tennessee,2014-2017,Poverty,133% to <400% FPL,51.4,44.0 to 58.8,342.0
4,Influenza,NaN,States/Local Areas,Tennessee,2014-2017,Poverty,<133% FPL,39.0,31.9 to 46.1,298.0


In [91]:
# pivot 'Dimension Type' into new columns (with 'Dimension' containing values for new columns)
df_vacc = pd.pivot(df_vacc, 
                   index=['Vaccine', 'Dose', 'Geography Type', 'Geography', 'Birth Year/Birth Cohort', \
                          'Dimension', 'Estimate (%)', '95% CI (%)', 'Sample Size'], # identifier for each row
                   columns=['Dimension Type'], # column containing new column names
                   values='Dimension')         # values for new columns

df_vacc.reset_index(inplace=True) 
df_vacc.rename_axis(None, axis=1, inplace=True) # remove the named index

In [92]:
# drop 'Dimension' column as it is no longer needed after pivot
df_vacc.drop(columns='Dimension', inplace = True)

In [93]:
# split 'Birth Year/Birth Cohort' column into two columns by looking for hyphen in values
df_vacc['Birth Year'] = np.where(df_vacc['Birth Year/Birth Cohort']\
                                 .str.contains('-'), np.nan, df_vacc['Birth Year/Birth Cohort'])
df_vacc['Birth Cohort'] = np.where(df_vacc['Birth Year/Birth Cohort']\
                                   .str.contains('-'), df_vacc['Birth Year/Birth Cohort'], np.nan)

In [94]:
# drop 'Birth Year/Birth Cohort' column as it is no longer needed after split
df_vacc.drop(columns='Birth Year/Birth Cohort', inplace = True)

In [95]:
# check for NA in Estimate (%) which will be column needed in all data visualizations in app
df_vacc['Estimate (%)'].isna().sum()

58

In [96]:
# drop rows with NA in Estimate (%) which will be column needed in all data visualizations in app
df_vacc.dropna(subset=['Estimate (%)'], inplace=True)

num_rows_current = df_vacc.shape[0]

print(num_rows_original - num_rows_current, "rows dropped")

58 rows dropped


In [97]:
# split 95% CI column into upper and lower bounds for possible use in visualizations
df_vacc['95% CI Lower (%)'] = df_vacc['95% CI (%)'].str.split(' to ').str[0]
df_vacc['95% CI Upper (%)'] = df_vacc['95% CI (%)'].str.split(' to ').str[1]

In [98]:
# fill NA values for Dose for certain vaccines (based on CDC documentation of what these represent)
df_vacc['Dose'] = np.where(df_vacc['Vaccine'] == 'Combined 7 Series', 'Full Series', df_vacc['Dose'])
df_vacc['Dose'] = np.where(df_vacc['Vaccine'] == 'Influenza', '≥2 Doses at least 24 days apart', df_vacc['Dose'])
df_vacc['Dose'] = np.where(df_vacc['Vaccine'] == 'Rotavirus', 'Full Series', df_vacc['Dose'])

In [99]:
# update values for HHS Regions to actually include 'HHS' in value to be more clear what Region means
df_vacc['Geography'] = np.where(df_vacc['Geography'].str.contains('Region'), \
                                df_vacc['Geography'].str.replace('Region', 'HHS Region'), df_vacc['Geography'])

In [100]:
# rename several columns to be more clear for use in app
df_vacc.rename(columns={'Geography': 'Geographic Area', 'Poverty': 'Poverty Level', \
                        'Insurance Coverage': 'Health Insurance Coverage'}, inplace=True)

In [101]:
# will add column to convert state names to abbreviations for use in choropleth map visualization
# dictionary to assist with conversion
state_abbrev = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR", "California": "CA", "Colorado": "CO", \
    "Connecticut": "CT", "Delaware": "DE", "Florida": "FL", "Georgia": "GA", "Hawaii": "HI", "Idaho": "ID", \
    "Illinois": "IL", "Indiana": "IN", "Iowa": "IA", "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA", \
    "Maine": "ME", "Maryland": "MD", "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN", \
    "Mississippi": "MS", "Missouri": "MO", "Montana": "MT", "Nebraska": "NE", "Nevada": "NV", \
    "New Hampshire": "NH", "New Jersey": "NJ", "New Mexico": "NM", "New York": "NY", "North Carolina": "NC", \
    "North Dakota": "ND", "Ohio": "OH", "Oklahoma": "OK", "Oregon": "OR", "Pennsylvania": "PA", \
    "Rhode Island": "RI", "South Carolina": "SC", "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX", \
    "Utah": "UT", "Vermont": "VT", "Virginia": "VA", "Washington": "WA", "West Virginia": "WV", \
    "Wisconsin": "WI", "Wyoming": "WY"}

In [102]:
# get list of state names from dictionary
states = list(state_abbrev.keys())

# Step 1: if Geographic Area is in list of states, then add as value in new State column
df_vacc['State'] = np.where(df_vacc['Geographic Area'].isin(states), df_vacc['Geographic Area'], np.nan)

# Step 2: use dictionary to replace each state name (key) with its state abbreviation (value)
df_vacc['State'] = df_vacc['State'].str.strip().replace(state_abbrev)

In [103]:
# reorder columns
df_vacc = df_vacc[['Vaccine', 'Dose', 'Geography Type', 'Geographic Area', 'State', 'Birth Year', 'Birth Cohort', \
                   'Estimate (%)', '95% CI (%)', '95% CI Lower (%)', '95% CI Upper (%)', 'Age', 'Race and Ethnicity', \
                    'Poverty Level', 'Health Insurance Coverage', 'Urbanicity', 'Overall', 'Sample Size']]

In [118]:
df_vacc.sample(10)

,Vaccine,Dose,Geography Type,Geographic Area,State,Birth Year,Birth Cohort,Estimate (%),95% CI (%),95% CI Lower (%),95% CI Upper (%),Age,Race and Ethnicity,Poverty Level,Health Insurance Coverage,Urbanicity,Overall,Sample Size
46567,Hep B,≥3 Doses,States/Local Areas,Ohio,OH,NaN,2015-2016,89.9,86.5 to 92.7,86.5,92.7,24 Months,NaN,NaN,NaN,NaN,NaN,575.0
20853,Hep A,≥1 Dose,States/Local Areas,California,CA,NaN,2013-2014,84.1,79.3 to 88.4,79.3,88.4,24 Months,NaN,NaN,NaN,NaN,NaN,674.0
36703,Hep B,"≥1 Dose, 3 Day (Birth Dose)",States/Local Areas,South Carolina,SC,2012,NaN,72.4,64.2 to 79.3,64.2,79.3,0-3 Days,NaN,NaN,NaN,NaN,NaN,255.0
48246,Hep B,≥3 Doses,States/Local Areas,Texas,TX,2019,NaN,64.3,58.6 to 69.6,58.6,69.6,7 Months,NaN,NaN,NaN,NaN,NaN,919.0
50086,Hib,Full Series,States/Local Areas,Colorado,CO,NaN,2018-2019,95.0,92.1 to 96.9,92.1,96.9,19 Months,NaN,NaN,NaN,NaN,NaN,561.0
19513,DTaP,≥4 Doses,States/Local Areas,Vermont,VT,NaN,2016-2019,89.0,85.6 to 92.4,85.6,92.4,NaN,NaN,133% to <400% FPL,NaN,NaN,NaN,522.0
75360,Influenza,≥2 Doses at least 24 days apart,States/Local Areas,TX-Dallas County,NaN,NaN,2016-2019,73.5,61.1 to 85.9,61.1,85.9,NaN,"White, Non-Hispanic",NaN,NaN,NaN,NaN,66.0
86060,PCV,≥3 Doses,HHS Regions/National,United States,NaN,NaN,2014-2017,78.2,74.9 to 81.5,74.9,81.5,NaN,NaN,NaN,Uninsured,NaN,NaN,1692.0
18984,DTaP,≥4 Doses,States/Local Areas,TX-City of Houston,NaN,NaN,2014-2015,83.8,78.3 to 88.6,78.3,88.6,24 Months,NaN,NaN,NaN,NaN,NaN,295.0
79698,MMR,≥1 Dose,States/Local Areas,NY-Rest of state,NaN,2012,NaN,94.2,89.0 to 97.4,89.0,97.4,35 Months,NaN,NaN,NaN,NaN,NaN,249.0


In [119]:
# saved cleaned data to csv
df_vacc.to_csv('child_vaccination_data_cleaned.csv', index=False)

In [120]:
# export set of unique values for Geographic Area which will be manipulated to create preferred sort order for filter in app
# Preferred Order = US first, then HHS Regions by number, then States alphabetically with City/County ordered after their state
geo_areas = df_vacc['Geographic Area'].unique()
df_geo_areas = pd.DataFrame(geo_areas)
df_geo_areas.to_csv('geo_areas.csv', index=False)